Or just flesh out ssa is functional programming
mimir

https://c9x.me/compile/doc/il.html
https://pypy.org/posts/2024/07/toy-abstract-interpretation.html
https://gist.github.com/tekknolagi/4425b28d5267e7bae8b0d7ef8fb4a671

Yea man, I need a compiler project.

https://dl.acm.org/doi/abs/10.1145/3689717 A Low-Level Look at A-Normal Form

Convert qbe to block args form


A lowkey branching.
No yea, that ought to be possible.
If we have direct jmps to constants.






In [ ]:
def blocks(start_addr):
    blocks = {}
    todo = [start_addr]
    while todo:
        addr = todo.pop()
        block = blocks[addr]
        insn = addr
        while True:
            insn = ctx.translate(insn)
            block.append(insn)
            if insn.is_branch():
                if not_constant():
                    # Todo. Do symbolic execution here?
                    # Do a value propagation?
                    # I'm like reimplementing angr... is that ok?
                    raise Exception("Indirect branch. More careful analysis needed.")
                else:







In [66]:
import lark
from kdrag.all import *
from kdrag.contrib.ir import *
grammar = r"""
start : NL* funcdef NL*
funcdef: "function" GLOBAL "(" [param_list] ")" NL? "{" NL block+ "}"
block: LABEL "(" [param_list] ")" NL instr* jump NL
instr: TEMP "=" BASETY OP operand ("," operand)* NL

jump: call | ite | ret

ite : "ite" operand "," call "," call
// jmp: "jmp" call
//jnz: ("jnz" | "ite") val "," call "," call
ret: "ret" [operand] // make a call?

call : LABEL "(" [param_list] ")"

?operand: SIGNED_INT | TEMP | GLOBAL
OP: /[A-Za-z][A-Za-z0-9]+/

param_list: param ("," param)*
param : BASETY TEMP

GLOBAL: /\$[A-Za-z_.][A-Za-z0-9_.]*/
LABEL: /@[A-Za-z_.][A-Za-z0-9_.]*/
TEMP: /%[A-Za-z_.][A-Za-z0-9_.]*/
BASETY: "w" | "l" | "s" | "d" | "m" // add m for memory

%import common.WS_INLINE
%import common.NEWLINE
%import common.ESCAPED_STRING
%import common.SIGNED_INT
%ignore WS_INLINE
%ignore /#[^\n]*/
NL: NEWLINE+
"""
PARSER = lark.Lark(grammar, start="start", parser="lalr")

example = r"""
function $foo(m %mem){
@entry(w %x, w %y)
    %t =w bvadd %x, %y 
    ret %mem
@loop(d %mem)
    %t =w bvneg %x, %y
    @entry(w %t)
}
"""
PARSER.parse(example)

basety = {
    "w" : smt.BitVecSort(32),
    "l" : smt.BitVecSort(64),
    "m" : smt.ArraySort(smt.BitVecSort(64), smt.BitVecSort(8)),
}

@lark.v_args(inline=True)
class FunctionTransformer(lark.Transformer):
    def __init__(self):
        self.labels : dict[str, smt.FuncDeclRef] = {}
        self.temps : dict[str, smt.ExprRef] = {}
        self.funcs : dict[str, smt.FuncDeclRef] = {}
    def funcdef(self, blocks):
        return Function(blocks)
    def block(self, label, params, nl1, insns, jmp, nl2):
        print("params", params, "insn", insns, "jm", jmp)
        return (label, Block(params, insns + [jmp]))
    def call(self, label, params): ...
    def BASETY(self, ty):
        return basety[ty.value]
    def param(self, ty, name):
        self.temps[name] = smt.Const(name, ty)
        return smt.Const(name, ty)
    def param_list(self, *params):
        return list(params)
    def operand(self, op):
        if isinstance(op, lark.Token):
            if op.type == "SIGNED_INT":
                return smt.BitVecVal(int(op.value), 32)
            elif op.type == "TEMP":
                print(self.temp[op.value])
                return self.temps[op.value]
            elif op.type == "GLOBAL":
                return smt.Const(op.value, smt.BitVecSort(64))
        else:
            raise Exception("Unknown operand type")
    def instr(self, dest, ty, op, *operands):
        print(operands)
        c = smt.Const(dest.value, ty)
        self.temps[dest] = c
        if op == "bvadd":
            expr = operands[0] + operands[1]
        elif op == "bvneg":
            expr = -operands[0]
        else:
            raise Exception(f"Unknown op {op}")
        return expr
    def NL(self, token):
        return None
    #def TEMP(self, name):
    #    #print(name)
    #    #return self.temps[name]


def parse(s : str):
    tree = PARSER.parse(s)
    builder = FunctionTransformer()
    return builder.transform(tree)

parse(example)


(Token('TEMP', '%x'), Token('TEMP', '%y'), None)
params [%x, %y] insn %x%y jm Tree(Token('RULE', 'jump'), [Tree(Token('RULE', 'ret'), [Token('TEMP', '%mem')])])


VisitError: Error trying to process rule "block":

can only concatenate str (not "list") to str

In [ ]:
def render_qbe(b : Block):
    # Convert to Phi nodes
    phis =[[] in range(b.arity)]
    for pred in preds(b):
        jmp = pred.insns[-1]
        if smt.is_ite(jmp):
        else:
            if jmp.decl() == self:
                for xs, c in zip(phis, jmp.children)
                    xs.append(c)
        



    for insn in b.insns:
        


In [ ]:
import operator
ops = {
    "bvadd", operator.add,
    "add" : operator.add,
    "sub" : operator.sub,
    ""
}

In [24]:
%%file /tmp/swap.s

.global swap
swap:
    mov %rdi, %rax
    mov %rsi, %rdi
    mov %rax, %rsi
    ret

Writing /tmp/swap.s


In [25]:
!gcc -c -o /tmp/swap /tmp/swap.s

In [ ]:
import angr

proj = angr.Project("/tmp/swap")
proj.loader.find_symbol("swap").rebased_addr
cfg = proj.analyses.CFG()
list(cfg.functions["swap"].blocks)
dict(proj.kb.functions)


WARNING  | 2026-01-28 11:59:16,933 | angr.analyses.cfg.cfg_base | "auto_load_libs" is enabled. With libraries loaded in project, CFG will cover libraries, which may take significantly more time than expected. You may reload the binary with "auto_load_libs" disabled, or specify "regions" to limit the scope of CFG recovery.


{4194304: <Function swap (0x400000)>}

In [ ]:
import kdrag.contrib.pcode as pcode
import kdrag.contrib.pcode.asmspec as asmspec

#help(pcode.BinaryContext)
ctx = pcode.BinaryContext("/tmp/swap")
func = ctx.function("swap")
ctx.block_of_addr("swap")

class Block():
    start_addr : int
    end_addr : int
    insns : list[pcode.Instruction]
    args : list[smt.ArrayRef] # mem, reg, unique
    rets : list[smt.ArrayRef] # mem, reg, unique
    jmps : list[tuple[smt.BoolRef, int]]


Unexpected SP conflict


AttributeError: 'BinaryContext' object has no attribute 'get_function_by_name'

In [7]:
import kdrag.contrib.ir as ir
from kdrag.all import *
help(ir.Block.of_expr)

Help on method of_expr in module kdrag.contrib.ir:

of_expr(e: z3.z3.ExprRef) -> 'Block' class method of kdrag.contrib.ir.Block
    >>> x,y = smt.BitVecs("x y", 64)
    >>> x,y = smt.Var(1, smt.BitVecSort(64)), smt.Var(0, smt.BitVecSort(64))
    >>> z = smt.BitVec("z", 64)
    >>> Block.of_expr(smt.If(True, (x + y)*42, x - y + z))
    ^(bv64,bv64):
    %0 = bvadd %var1, %var0
    %1 = bvmul %0, 42
    %2 = bvsub %var1, %var0
    %3 = bvadd %2, z
    %4 = if True, %1, %3



In [ ]:
from dataclasses import dataclass, field
from kdrag.all import *
class Block():
    name : str
    args : list[smt.ExprRef]
    insns : list[smt.ExprRef]
    jmp : smt.ExprRef # either if x, block1, block2 or just goto block

class Function():
    blocks : list[Block] # 0th block is entry. Or "entry" is entry? Naw. 0th.

Bottom = smt.DeclareSort("Bottom")
ret = smt.Function("ret", smt.BitVecSort(64), Bottom)


In [ ]:
x = smt.BitVec("x", 64)
(x+x).decl().name()


'bvadd'

In [8]:
smt.is_bv_value(x)

False

In [ ]:
from typing import Union
@dataclass
class LetBlock():
    defns : dict[smt.ExprRef, smt.ExprRef]
    body : Union[smt.ExprRef,"LetBlock"]
    def expand(self):
        if isinstance(self.body, LetBlock):
            body = self.body.expand()
            # Or maybe we should expand into the inner lets, taking care of capture avoidance?
        else:
            body = self.body
        return smt.substitute(body, *self.defns.items())

x,y = smt.Ints("x y")
LetBlock({x : smt.IntVal(3)}, x + 7).expand()


3 + 7

In [75]:
from dataclasses import dataclass, field
import pprint
Bottom = smt.DeclareSort("Bottom")
ret = smt.Function("ret", smt.BitVecSort(64), Bottom)
mem = smt.Array("mem", smt.BitVecSort(64), smt.BitVecSort(64))
@dataclass
class Block():
    args : list[smt.ExprRef]
    insns : list[smt.ExprRef] = field(default_factory=list)
    @classmethod
    def of_expr(cls, vs, e):
        insns = []
        seen = set(vs)
        todo = [e]
        while todo:
            e = todo.pop()
            insns.append(e)
            for arg in e.children():
                if arg not in seen:
                    seen.add(arg)
                    todo.append(arg)
        return cls(args=vs, insns=list(reversed(insns)))
    def __repr__(self):
        res = ["blk(" + ", ".join("%" + str(a.get_id()) for a in self.args) + "):"]
        for insn in self.insns:
            res.append(f"let %{insn.get_id()} := {insn.decl().name()} " + ", ".join("%" + str(c.get_id()) for c in insn.children()))
        #res.append(f"ret %{self.insns[-1].get_id()}")
        return "\n".join(res)
    def cstr(self):
        res = []
        for insn in self.insns:
            decl = insn.decl().name()
            children = [f"v{c.get_id()}" for c in insn.children()]
            if decl == "ret":
                res.append(f"return {children[0]};")
            elif decl == "bvadd":
                res.append(f"uint64_t v{insn.get_id()} = {children[0]} + {children[1]};")
            elif decl == "bvmul":
                res.append(f"uint64_t v{insn.get_id()} = {children[0]} * {children[1]};")
            elif decl == "ite":
                raise NotImplementedError("ite not implemented yet")
            else:
                res.append(f"uint64_t v{insn.get_id()} = {insn.decl().name()}(" + ", ".join(children) + ");")
        return "\n".join(res)
    def riscv(self):
        res = []
        pass

x = smt.BitVec("x", 64)
#x = smt.BitVecVal(42, 64)
b = Block.of_expr([x], ret(x + x + x*x))

@dataclass
class Function():
    blocks : dict[str, Block] = field(default_factory=dict)
    defns : dict[str, smt.FuncDeclRef] = field(default_factory=dict)
    def add_block(self, name : str, vs, body) -> smt.FuncDeclRef:
        assert name not in self.blocks
        f = kd.define(name, vs, body)
        self.defns[name] = f
        self.blocks[name] = Block.of_expr(vs, body)
        return f
    def cstr(self):
        res = []
        for name, blk in self.blocks.items():
            res.append(f"void {name}(" + ", ".join(f"uint64_t v{a.get_id()}" for a in blk.args) + ") {")
            res.append(blk.cstr())
            res.append("}")
        return "\n\n".join(res)
    def simp(self, *args):
        return kd.full_simp(kd.full_simp(self.defns["entry"](*args)))

m = Function()
last = m.add_block("last", [x], ret(x + x + x*x))
entry = m.add_block("entry", [x], last(x + x))


#print(b.cstr())
print(m.cstr())


sumn = Function()
acc, n = smt.BitVecs("acc n", 64)
done = sumn.add_block("done", [acc], ret(acc))
loop = smt.Function("loop", smt.BitVecSort(64), smt.BitVecSort(64), Bottom)
loop = sumn.add_block("loop", [acc, n], 
            smt.If(n > 0, loop(acc + n, n - 1), done(acc))
            )
entry = sumn.add_block("entry", [n], loop(smt.BitVecVal(0, 64), n)) 
pprint.pprint(sumn)

sumn.simp(10)



void last(uint64_t v345) {

uint64_t v497 = v345 + v345;
uint64_t v467 = v345 * v345;
uint64_t v474 = v497 + v467;
return v474;

}

void entry(uint64_t v345) {

uint64_t v497 = v345 + v345;
uint64_t v444 = last(v497);

}
Function(blocks={'done': blk(%488):
let %507 := ret %488,
                 'entry': blk(%468):
let %469 := bv 
let %462 := loop %469, %468,
                 'loop': blk(%488, %468):
let %469 := bv 
let %513 := bvsgt %468, %469
let %516 := bvadd %488, %468
let %508 := bv 
let %509 := bvsub %468, %508
let %495 := loop %516, %509
let %503 := done %488
let %501 := if %513, %495, %503},
         defns={'done': done, 'entry': entry, 'loop': loop})


ret(55)

In [ ]:
type Loc = str
type Insn = str
def compile_riscv(blk : Block, locs : dict[smt.ExprRef, Loc]):
    res : list[Insn] = []
    for insn in reversed(blk.insns):
        if smt.is_bv_value(insn):
            loc = alloc(x, locs)
            res.append(f"li {loc} {insn.as_long()}")
        else:
            decl = insn.decl().name()
            children = insn.children()
            [alloc(c, locs) for c in children]
            match decl:
                case "bvadd":
                    res.append(f"add {} {}")
                case "bvsub":
                    res.append(f"sub {} {}")
                case "bvmul":
                    res.append(f"mul {} {}")
        if insn in locs:
            del insn
        if is_add
    res.reverse()
Bottom = smt.DeclareSort("Bottom")
ret = smt.Function("ret", smt.BitVecSort(64), Bottom)

[
    x := smt.BitVecVal(42),
    y := x + x,
    z := y * y,
    ret(z)
]

SyntaxError: f-string: valid expression required before '}' (2845523921.py, line 13)

The similarly of a malloc implementation and this alloc are interesting.
We need to alloc going backwards when something is used, and free it when it is defined.


In [ ]:
@dataclass
class Alloc():
    regs : set[str]
    stack : int = 0
    def alloc(self): # take program as argument? Take size as argument?
        if self.regs:
            return self.regs.pop()
        else:
            self.stack += 8
            return f"sp+{self.stack}"
    def free(self, ref):
        if ref.startswith("sp+"):
            # don't free stack space
            # happens at the end
            pass
        else:
            self.regs.add(ref)

def riscv64():
    return Alloc(set(["t0","t1","t2","t3","t4","t5","t6"]))

class CallConv():
    args : list[str]

{"ret" : CallConv(["r1"])}


class SymAlloc():
    live : set(smt.ExprRef)
    stack : smt.ArithRef = smt.IntVal(0)
    constrs : list[smt.BoolRef]
    def alloc(self, sort) -> smt.ExprRef:
        v = 
        self.live.add(smt.)
        self.constraints(smt.Distinct(*self.live))
    def free(self, v : smt.ExprRef):
        self.live.remove(v)

In [ ]:

def alloc(x, locs):
    if x in locs:
        return locs[x]
    else:
        for reg in [f"r{n}" for n in range(32)]:
            if reg not in locs.values():
                return reg
        for i in range(1000):
            raise Exception("ran out of registers")
    

def compile_riscv(blk : Block, locs):
    res = []
    for insn in reversed(blk.insns):
        if smt.is_bv_value(insn):
            loc = alloc(x, locs)
            res.append(f"li {loc} {insn.as_long()}")
        else:
            decl = insn.decl().name()
            children = insn.children()
            [alloc(c, locs) for c in children]
            match decl:
                case "bvadd":
                    res.append(f"add {} {}")
                case "bvsub":
                    res.append(f"sub {} {}")
                case "bvmul":
                    res.append(f"mul {} {}")
                case "store":
                    res.append(f"sw {} {}({})")
                case "select":
                    res.append(f"lw {} {}({})")
        if insn in locs:
            del insn
        if is_add
    res.reverse()
    return res

def riscv_compile(f : Function):
    for b in reversed(blocks):
        


In [ ]:
def compile_riscv(insns : ):
    for insn in reversed(insns):


In [ ]:

@dataclass 
class Call(): # App
    f : str
    args : tuple[Expr, ...]

@dataclass
class Defn():
    name : str
    nargs : int
    body : Expr